In [1]:
import sys

sys.path.append('../GTS/GTS')

In [2]:
import torch
import numpy as np
import pandas as pd
import os
import time
from glob import glob
from itertools import cycle

In [3]:
dirs = '../M5_Dataset'

In [4]:
INPUT_DIR = dirs

calendar = pd.read_csv(f'{INPUT_DIR}/calendar.csv')
calendar.name = 'calendar'

prices = pd.read_csv(f'{INPUT_DIR}/sell_prices.csv')
prices.name = 'prices'

sales = pd.read_csv(f'{INPUT_DIR}/sales_train_evaluation.csv')
sales.name = 'sales'

In [24]:
def downcast(df):
    cols = df.dtypes.index.tolist()
    types = df.dtypes.values.tolist()
    for i,t in enumerate(types):
        if 'int' in str(t):
            if df[cols[i]].min() > np.iinfo(np.int8).min and df[cols[i]].max() < np.iinfo(np.int8).max:
                df[cols[i]] = df[cols[i]].astype(np.int8)
            elif df[cols[i]].min() > np.iinfo(np.int16).min and df[cols[i]].max() < np.iinfo(np.int16).max:
                df[cols[i]] = df[cols[i]].astype(np.int16)
            elif df[cols[i]].min() > np.iinfo(np.int32).min and df[cols[i]].max() < np.iinfo(np.int32).max:
                df[cols[i]] = df[cols[i]].astype(np.int32)
            else:
                df[cols[i]] = df[cols[i]].astype(np.int64)
        elif 'float' in str(t):
            if df[cols[i]].min() > np.finfo(np.float16).min and df[cols[i]].max() < np.finfo(np.float16).max:
                df[cols[i]] = df[cols[i]].astype(np.float16)
            elif df[cols[i]].min() > np.finfo(np.float32).min and df[cols[i]].max() < np.finfo(np.float32).max:
                df[cols[i]] = df[cols[i]].astype(np.float32)
            else:
                df[cols[i]] = df[cols[i]].astype(np.float64)
        elif t == object:
            if cols[i] == 'date':
                df[cols[i]] = pd.to_datetime(df[cols[i]], format='%Y-%m-%d')
            else:
                df[cols[i]] = df[cols[i]].astype('category')
    return df  

sales = downcast(sales)
prices = downcast(prices)
calendar = downcast(calendar)

In [26]:
df = sales.groupby(['store_id', 'dept_id']).sum()

index_list = df.index

df.reset_index(inplace=True)


idx_list = []
state_list = []
cat_list = []

for i in range(len(index_list)):
    idx = index_list[i][0] + '_' + index_list[i][1]
    idx_list.append(idx)
    
    state = df.iloc[i]['store_id'].split('_')
    state_list.append(state[0])
    
    cat = df.iloc[i]['dept_id'].split('_')
    cat_list.append(cat[0])
    
df['index'] = idx_list
df.set_index('index', inplace=True)

In [40]:
df['state_id'] = state_list
df['cat_id'] = cat_list

<ipython-input-40-f7269c669b7c>:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider using pd.concat instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  df['state_id'] = state_list
<ipython-input-40-f7269c669b7c>:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider using pd.concat instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  df['cat_id'] = cat_list


In [41]:
df

,store_id,dept_id,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,...,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941,state_id,cat_id
index,,,,,,,,,,,,,,,,,,,,,
CA_1_FOODS_1,CA_1,FOODS_1,297,284,214,175,182,191,224,263,...,415,277.0,302.0,397.0,330.0,310.0,427,334.0,CA,FOODS
CA_1_FOODS_2,CA_1,FOODS_2,674,655,396,476,354,486,554,747,...,781,555.0,476.0,427.0,411.0,434.0,627,719.0,CA,FOODS
CA_1_FOODS_3,CA_1,FOODS_3,2268,2198,1398,1607,1496,1730,1915,2952,...,3521,2366.0,2016.0,1992.0,2113.0,2249.0,2729,3274.0,CA,FOODS
CA_1_HOBBIES_1,CA_1,HOBBIES_1,528,489,409,383,263,453,339,750,...,631,389.0,347.0,353.0,354.0,494.0,656,588.0,CA,HOBBIES
CA_1_HOBBIES_2,CA_1,HOBBIES_2,28,9,6,9,5,14,7,19,...,56,28.0,46.0,28.0,52.0,45.0,63,58.0,CA,HOBBIES
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
WI_3_FOODS_3,WI_3,FOODS_3,2293,2383,1841,1965,1427,2760,2727,3786,...,3162,1952.0,1602.0,1594.0,1627.0,2011.0,2360,2632.0,WI,FOODS
WI_3_HOBBIES_1,WI_3,HOBBIES_1,256,342,228,183,70,285,357,399,...,294,187.0,174.0,287.0,220.0,242.0,273,305.0,WI,HOBBIES
WI_3_HOBBIES_2,WI_3,HOBBIES_2,22,14,20,11,4,8,3,13,...,38,11.0,45.0,45.0,46.0,27.0,31,37.0,WI,HOBBIES


In [19]:
df_melt = pd.melt(df, id_vars=['store_id', 'dept_id', 'state_id', 'cat_id'], var_name='d', value_name='sold').dropna()

In [20]:
df_melt

,store_id,dept_id,state_id,cat_id,d,sold
0,CA_1,FOODS_1,CA,FOODS,d_1,297.0
1,CA_1,FOODS_2,CA,FOODS,d_1,674.0
2,CA_1,FOODS_3,CA,FOODS,d_1,2268.0
3,CA_1,HOBBIES_1,CA,HOBBIES,d_1,528.0
4,CA_1,HOBBIES_2,CA,HOBBIES,d_1,28.0
...,...,...,...,...,...,...
135865,WI_3,FOODS_3,WI,FOODS,d_1941,2632.0
135866,WI_3,HOBBIES_1,WI,HOBBIES,d_1941,305.0
135867,WI_3,HOBBIES_2,WI,HOBBIES,d_1941,37.0
135868,WI_3,HOUSEHOLD_1,WI,HOUSEHOLD,d_1941,723.0


In [12]:
calendar

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,2011-01-29,11101,Saturday,1,1,2011,d_1,NaN,NaN,NaN,NaN,0,0,0
1,2011-01-30,11101,Sunday,2,1,2011,d_2,NaN,NaN,NaN,NaN,0,0,0
2,2011-01-31,11101,Monday,3,1,2011,d_3,NaN,NaN,NaN,NaN,0,0,0
3,2011-02-01,11101,Tuesday,4,2,2011,d_4,NaN,NaN,NaN,NaN,1,1,0
4,2011-02-02,11101,Wednesday,5,2,2011,d_5,NaN,NaN,NaN,NaN,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1964,2016-06-15,11620,Wednesday,5,6,2016,d_1965,NaN,NaN,NaN,NaN,0,1,1
1965,2016-06-16,11620,Thursday,6,6,2016,d_1966,NaN,NaN,NaN,NaN,0,0,0
1966,2016-06-17,11620,Friday,7,6,2016,d_1967,NaN,NaN,NaN,NaN,0,0,0
1967,2016-06-18,11621,Saturday,1,6,2016,d_1968,NaN,NaN,NaN,NaN,0,0,0


In [15]:
prices

,store_id,item_id,wm_yr_wk,sell_price
0,CA_1,HOBBIES_1_001,11325,9.578125
1,CA_1,HOBBIES_1_001,11326,9.578125
2,CA_1,HOBBIES_1_001,11327,8.257812
3,CA_1,HOBBIES_1_001,11328,8.257812
4,CA_1,HOBBIES_1_001,11329,8.257812
...,...,...,...,...
6841116,WI_3,FOODS_3_827,11617,1.000000
6841117,WI_3,FOODS_3_827,11618,1.000000
6841118,WI_3,FOODS_3_827,11619,1.000000
6841119,WI_3,FOODS_3_827,11620,1.000000


In [21]:
df_concat_calendar = pd.merge(df_melt, calendar, on='d', how='left')

In [22]:
df_concat_calendar

,store_id,dept_id,state_id,cat_id,d,sold,date,wm_yr_wk,weekday,wday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,CA_1,FOODS_1,CA,FOODS,d_1,297.0,2011-01-29,11101,Saturday,1,1,2011,NaN,NaN,NaN,NaN,0,0,0
1,CA_1,FOODS_2,CA,FOODS,d_1,674.0,2011-01-29,11101,Saturday,1,1,2011,NaN,NaN,NaN,NaN,0,0,0
2,CA_1,FOODS_3,CA,FOODS,d_1,2268.0,2011-01-29,11101,Saturday,1,1,2011,NaN,NaN,NaN,NaN,0,0,0
3,CA_1,HOBBIES_1,CA,HOBBIES,d_1,528.0,2011-01-29,11101,Saturday,1,1,2011,NaN,NaN,NaN,NaN,0,0,0
4,CA_1,HOBBIES_2,CA,HOBBIES,d_1,28.0,2011-01-29,11101,Saturday,1,1,2011,NaN,NaN,NaN,NaN,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135865,WI_3,FOODS_3,WI,FOODS,d_1941,2632.0,2016-05-22,11617,Sunday,2,5,2016,NaN,NaN,NaN,NaN,0,0,0
135866,WI_3,HOBBIES_1,WI,HOBBIES,d_1941,305.0,2016-05-22,11617,Sunday,2,5,2016,NaN,NaN,NaN,NaN,0,0,0
135867,WI_3,HOBBIES_2,WI,HOBBIES,d_1941,37.0,2016-05-22,11617,Sunday,2,5,2016,NaN,NaN,NaN,NaN,0,0,0
135868,WI_3,HOUSEHOLD_1,WI,HOUSEHOLD,d_1941,723.0,2016-05-22,11617,Sunday,2,5,2016,NaN,NaN,NaN,NaN,0,0,0
